<a href="https://colab.research.google.com/github/xxcramseyxx/GAN_image_creator/blob/main/GAN_image_creator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
#"""
#Created on Tue Apr  6 16:43:14 2021

#@author: xtianramses
#"""

from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
from torch.autograd import Variable

#setting sme of the hyperparameters
batchSize = 64
imageSize = 64

#creating the transformmations
transform = transforms.Compose([transforms.Resize(imageSize), transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5),(0.5, 0.5,  0.5)),])

#koading the dataset
dataset = dset.CIFAR10(root = '.\data', download = True, transform = transform)
dataloader = torch.utils.data.DataLoader(dataset, batch_size = batchSize, shuffle = True, num_workers= 0)

# Defining the weights_init function that takes as input a neural network m and that will initialize all its weights.
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        m.weight.data.normal_(0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)
        
#Defining the generator
class G(nn.Module):
    
    def __init__(self):
        super(G, self).__init__()        #super activates inheritance
        self.main = nn.Sequential(
                nn.ConvTranspose2d(100,512, 4, 1, 0, bias= False), #100=size of input, 512 num of feature maps output, 4=kernal size 4x4, 1 = stride, 0=padding
                nn.BatchNorm2d(512), #512= feature maps
                nn.ReLU(True), #arg inplace default is False
                nn.ConvTranspose2d(512, 256, 4, 2, 1, bias = False), #512= inputs from last cell, 256= new outputs  
                nn.BatchNorm2d(256), #256= feature maps
                nn.ReLU(True), #arg inplace default is False
                nn.ConvTranspose2d(256, 128, 4, 2, 1, bias = False),
                nn.BatchNorm2d(128),
                nn.ReLU(True),
                nn.ConvTranspose2d(128, 64, 4, 2, 1, bias = False),
                nn.BatchNorm2d(64),
                nn.ReLU(True),
                nn.ConvTranspose2d(64, 3, 4, 2, 1, bias = False),
                nn.Tanh()
        )
        
    def forward(self, input): #random input vector to generate noise to make a picture
        output = self.main(input)
        return output
    
#Creating the Generator
netG = G()
netG.apply(weights_init)

#Defining the Discriminator
class D(nn.Module):
        
    def __init__(self):
        super(D, self).__init__()
        self.main = nn.Sequential(
                nn.Conv2d(3, 64, 4, 2, 1, bias = False),
                nn.LeakyReLU(0.2, inplace = True), #0.2=neg slope, inplace activates the activation function
                nn.Conv2d(64, 128, 4, 2, 1, bias = False),
                nn.BatchNorm2d(128),
                nn.LeakyReLU(0.2, inplace =  True),
                nn.Conv2d(128, 256, 4, 2, 1, bias = False),
                nn.BatchNorm2d(256),
                nn.LeakyReLU(0.2, inplace = True),
                nn.Conv2d(256, 512, 4, 2, 1, bias = False),
                nn.BatchNorm2d(512),
                nn.Sigmoid()    
        )
        
    def forward(self, input):
        output = self.main(input)
        return output.view(-1) #.view(-1) flatens the convolution
        
#Creating the Discriminator
netD = D()
netD.apply(weights_init)
    
#Training he DCGANs
criterion = nn.BCELoss()
optimizerD = optim.Adam(netD.parameters(), lr = 0.0002, betas = (0.5, 0.999))
optimizerG = optim.Adam(netG.parameters(), lr = 0.0002, betas = (0.5, 0.999))  

for epoch in range(25):
    for i, data in enumerate(dataloader, 0):
      
        #1st step: Updating the weights of the Discriminator
        
        netD.zero_grad()
        
        #Training the Discriminator with a real image of the dataset
        
        real, _ = data
        input = Variable(real)
        target = Variable(torch.ones(input.size()[0]))
        output = netD(input)
        errD_real  = criterion(output, target)
        
        #Training the Discriminator with a fake image generated by the generator
        
        noise = Variable(torch.randn(input.size()[0], 100, 1, 1)) #100 = feature maps, 1 ,1 = of 1x1
        fake = netG(noise)
        target = Variable(torch.zeros(input.size()[0]))
        output = netD(fake.detatch()) #won't be a part of the SGD
        errD_fake  = criterion(output, target)
        
        #Backpropagating this total error
        errD = errD_real + errD_fake
        errD.backward()
        optimizerD.step()
    
        #2nd step: Updating the weightsf the neural network of the generator
        
        netG.zero_grad()
        target = Variable(torch.ones(input.size()[0]))
        output = netD(fake)
        errG = criterion(output, target)
        errG.backward()
        optimizerG.step()
        
        #3rd step: rinting the losses nd saving the real images and the generated images
        print('[%d/%d][%d/%d] Loss_D: %.4f Loss_G: %.4f' % (epoch, 25, i, len(dataloader), errD.data[0], errG.data[0])) # We print les losses of the discriminator (Loss_D) and the generator (Loss_G).
        if i % 100 == 0: # Every 100 steps:
            vutils.save_image(real, '%s/real_samples.png' % "./results", normalize = True) # We save the real images of the minibatch.
            fake = netG(noise) # We get our fake generated images.
            vutils.save_image(fake.data, '%s/fake_samples_epoch_%03d.png' % ("./results", epoch), normalize = True) # We also save the fake generated images of the minibatch.





